In [1]:
import json
import ast

import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
from tqdm.auto import tqdm
from scipy.stats import levene
import scipy.stats as st
import pylab
import statsmodels.stats.api as sms
import phik

import my_hypotheses as hyp
import LichessAnalys as li
import my_message as ms

c:\Program Files\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
lichessAnalys = li.LichessAnalys()
hypotheses = hyp.ProgressivePlayerCanBeACheater()
message_to_send = ms.MessageToSend()

In [3]:
df_for_test_group = pd.read_csv('./move_score_and_clocks.csv', sep=',')

In [4]:
df_for_control_group = \
     pd.read_csv('./merge_eval_and_clocks_for_control_group.csv', sep=',')

In [33]:
df_for_test_group

,date,game_id,time_control,clocks_list,move_count,user_id,move_score,clocks_list_new,move_score_new
0,2023-05-01,E40CHV0V,90+30,"[2.72, 1.68, 1.28, 0.88, 2.08, 0.48, 0.96, 0.5...",42,artistendo,"[36, 9, -26, -28, -26, -18, -5, -25, -10, -7, ...","[2.72, 1.68, 1.28, 0.88, 2.08, 0.48, 0.96, 0.5...","[36, 9, -26, -28, -26, -18, -5, -25, -10, -7, ..."
1,2023-04-30,EdUSUizY,90+30,"[0.56, 1.28, 1.2, 1.6, 5.76, 1.28, 1.36, 3.28,...",32,artistendo,"[36, 40, 74, 59, 33, 24, 30, 48, 69, -9, -38, ...","[0.56, 1.28, 1.2, 1.6, 5.76, 1.28, 1.36, 3.28,...","[36, 40, 74, 59, 33, 24, 30, 48, 69, -9, -38, ..."
2,2023-04-25,dzZQMB8X,90+30,"[5.04, 5.92, 6.32, 5.92, 29.04, 20.56, 14.24, ...",28,artistendo,"[36, 68, 47, 64, 81, 36, 55, 136, 158, 149, 15...","[5.04, 5.92, 6.32, 5.92, 29.04, 20.56, 14.24, ...","[36, 68, 47, 64, 81, 36, 55, 136, 158, 149, 15..."
3,2023-04-22,oUAgcjiN,90+30,"[1.68, 1.44, 1.12, 22.24, 4.08, 356.16, 19.36,...",41,artistendo,"[36, 9, -26, 0, 0, 0, 0, 9, 12, 67, 148, 155, ...","[1.68, 1.44, 1.12, 22.24, 4.08, 356.16, 19.36,...","[36, 9, -26, 0, 0, 0, 0, 9, 12, 67, 148, 155, ..."
4,2023-04-19,bQZff8t4,90+30,"[1.76, 0.0, 20.32, 9.92, 4.88, 4.48, 3.2, 0.56...",32,artistendo,"[36, 9, -26, -28, -40, -40, -5, -26, -15, -3, ...","[1.76, 0.0, 20.32, 9.92, 4.88, 4.48, 3.2, 0.56...","[36, 9, -26, -28, -40, -40, -5, -26, -15, -3, ..."
5,2023-04-17,h0sbNGGK,90+30,"[20.64, 27.12, 7.6, 24.32, 10.72, 43.04, 194.8...",30,artistendo,"[36, 29, 13, 3, -39, -53, -27, 19, 56, 94, 111...","[20.64, 27.12, 7.6, 24.32, 10.72, 43.04, 194.8...","[36, 29, 13, 3, -39, -53, -27, 19, 56, 94, 111..."
6,2023-04-05,PObmYKgw,30+20,"[4.72, 4.64, 4.48, 3.68, 14.72, 10.64, 7.28, 5...",62,artistendo,"[36, 40, 74, 50, 0, 51, 32, 2, -24, -29, 8, 50...","[4.72, 4.64, 4.48, 3.68, 14.72, 10.64, 7.28, 5...","[36, 40, 74, 50, 0, 51, 32, 2, -24, -29, 8, 50..."
7,2023-03-31,9UbP1n9S,30+20,"[3.92, 14.24, 16.56, 7.12, 12.96, 0.64, 13.84,...",38,artistendo,"[36, 30, 17, 39, 33, 55, 27, 26, 59, 43, 52, 9...","[3.92, 14.24, 16.56, 7.12, 12.96, 0.64, 13.84,...","[36, 30, 17, 39, 33, 55, 27, 26, 59, 43, 52, 9..."
8,2023-03-31,0Na0ISgm,30+20,"[3.28, 3.12, 32.48, 20.88, 109.28, 69.6, 1.52,...",60,artistendo,"[23, 66, 88, 87, 82, 96, 85, 104, 108, 51, 81,...","[3.28, 3.12, 32.48, 20.88, 109.28, 69.6, 1.52,...","[23, 66, 88, 87, 82, 96, 85, 104, 108, 51, 81,..."
9,2023-03-06,gIp3K3P6,30+20,"[2.32, 6.4, 1.2, 1.36, 1.44, 3.12, 5.68, 104.6...",29,artistendo,"[36, 9, -26, 46, 44, 96, 87, 102, 81, -7, 8, 8...","[2.32, 6.4, 1.2, 1.36, 1.44, 3.12, 5.68, 104.6...","[36, 9, -26, 46, 44, 96, 87, 102, 81, -7, 8, 8..."


In [32]:
type(df_for_test_group['clocks_list'][0])

str

In [21]:
def add_correlation_coefficient(
    df_for_satatistical_test: pd.DataFrame) -> pd.Series:
    df_for_satatistical_test['clocks_list_new'] = \
    [ast.literal_eval(i) for i in df_for_satatistical_test['clocks_list']]
    df_for_satatistical_test['move_score_new'] = \
    [ast.literal_eval(i) for i in df_for_satatistical_test['move_score']]

    len_df = df_for_satatistical_test.shape[0]
    list_df_len = list(range(0, len_df, 1))
    correlation_list = []

    for i in list_df_len:
        len_clocks = len(df_for_satatistical_test['clocks_list_new'][i])
        len_score = len(df_for_satatistical_test['move_score_new'][i])
        if len_clocks != len_score:
            if len_clocks < len_score:
                df_for_satatistical_test['clocks_list_new'][i] \
                .append(df_for_satatistical_test['clocks_list_new'][i][-1])
            else:
                df_for_satatistical_test['move_score_new'][i] \
                .append(df_for_satatistical_test['move_score_new'][i][-1])
        else:
            d = {
            'clocks_list': df_for_satatistical_test['clocks_list_new'][i],
            'move_score': df_for_satatistical_test['move_score_new'][i]
            }
            df = pd.DataFrame(data=d)
            t = df[['clocks_list', 'move_score']]
            phik_overview = t.phik_matrix()
            correlation_coefficient = phik_overview['clocks_list'][1]
            correlation_list.append(correlation_coefficient)
    result = pd.Series(correlation_list)
    return result

In [22]:
add_correlation_coefficient = add_correlation_coefficient(
        df_for_satatistical_test=df_for_test_group)
add_correlation_coefficient \
    .to_csv('./add_correlation_coefficient.csv',
    index=False,
    sep=",")

interval columns not set, guessing: ['clocks_list', 'move_score']
interval columns not set, guessing: ['clocks_list', 'move_score']
interval columns not set, guessing: ['clocks_list', 'move_score']
interval columns not set, guessing: ['clocks_list', 'move_score']
interval columns not set, guessing: ['clocks_list', 'move_score']
interval columns not set, guessing: ['clocks_list', 'move_score']
interval columns not set, guessing: ['clocks_list', 'move_score']
interval columns not set, guessing: ['clocks_list', 'move_score']
interval columns not set, guessing: ['clocks_list', 'move_score']


In [23]:
print(add_correlation_coefficient)

0    0.000000
1    0.871462
2    0.754409
3    0.000000
4    0.000000
5    0.553973
6    0.654189
7    0.543104
8    0.626308
dtype: float64


In [24]:
def levene_test(df_for_control_group: pd.DataFrame,
    df_for_test_group: pd.DataFrame) -> pd.DataFrame:
    result = pd.DataFrame(columns=[
            'levene_p_value_list'])

    df_for_test_group['clocks_list_new'] = \
    [ast.literal_eval(i) for i in df_for_test_group['clocks_list']]
    df_for_control_group['clocks_list_new'] = \
    [ast.literal_eval(i) for i in df_for_control_group['clocks_list']]

    test_group = df_for_test_group['clocks_list_new']
    control_group = df_for_control_group['clocks_list_new']

    for i in test_group:
        levene_p_value_list = []
        for k in control_group:
            stat, p = levene(k, i, center='mean')
            levene_p_value_list.append(round(p, 3))
        columns = {'levene_p_value_list': str(levene_p_value_list)}
        df = pd.DataFrame(data=columns, index=[0])
        result = \
        pd.concat([result, \
            df], ignore_index=True , join="outer")

    result['levene_p_value_list'] = \
    [ast.literal_eval(i) for i in result['levene_p_value_list']]

    leven_median = []
    for i in result['levene_p_value_list']:
        leven_median.append(np.median(i))
    result['levene_p_value_median'] = leven_median
    result['game_id'] = df_for_test_group['game_id']
    result = result.drop(columns=['levene_p_value_list'])
    return result

In [25]:
levene_test = levene_test(
    df_for_control_group=df_for_control_group,
    df_for_test_group=df_for_test_group)
levene_test \
    .to_csv('./levene_test.csv',
    index=False,
    sep=",")

In [26]:
lev = pd.read_csv('./levene_test.csv',
    sep=",")
lev


,levene_p_value_median,game_id
0,0.0010,E40CHV0V
1,0.2130,EdUSUizY
2,0.0200,dzZQMB8X
3,0.0000,oUAgcjiN
4,0.0000,bQZff8t4
5,0.0000,h0sbNGGK
6,0.1110,PObmYKgw
7,0.0675,9UbP1n9S
8,0.0030,0Na0ISgm
9,0.0000,gIp3K3P6


In [27]:
user_id = 'artistendo'

In [28]:
def combining_main_and_statistical_data(
    user_id: str,
    levene_test: pd.DataFrame,
    merge_eval_and_clocks: pd.DataFrame,
    add_correlation_coefficient: pd.Series) -> pd.DataFrame:
    user_id = user_id
    df = merge_eval_and_clocks.query('user_id == @user_id')
    levene_test['correlation_coefficient']= add_correlation_coefficient
    result = df.merge(levene_test,
            on='game_id',
            how='left')
    return result

In [29]:
df_classical = pd.read_csv('./df_classical.csv', sep=',')

In [30]:
combining_main_and_statistical_data = combining_main_and_statistical_data(
    user_id=user_id,
    levene_test=levene_test,
    merge_eval_and_clocks=df_classical,
    add_correlation_coefficient=add_correlation_coefficient)
combining_main_and_statistical_data \
    .to_csv('./combining_main_and_statistical_data.csv',
    index=False,
    sep=",")

In [31]:
combining_main_and_statistical_data

,game_id,date,game_speed,rating,rating_diff,clocks_mean,clocks_std,clocks_median,move_count,time_control,user_id,mistake,blunder,inaccuracy,acpl,levene_p_value_median,correlation_coefficient
0,Ul1yrsZ4,2023-05-06,classical,2401,12,89.20,150.92,29.5,20,90+30,artistendo,0,0,3,17,NaN,NaN
1,nOmDQ4PW,2023-05-04,classical,2387,14,31.00,59.84,8.0,17,90+30,artistendo,0,2,1,62,NaN,NaN
2,E40CHV0V,2023-05-01,classical,2386,1,8.62,7.87,7.0,42,90+30,artistendo,1,2,4,28,0.0010,0.000000
3,EdUSUizY,2023-04-30,classical,2375,11,14.06,36.28,3.0,32,90+30,artistendo,1,3,2,62,0.2130,0.871462
4,dzZQMB8X,2023-04-25,classical,2388,-13,44.48,73.25,21.0,28,90+30,artistendo,1,1,3,42,0.0200,0.754409
5,oUAgcjiN,2023-04-22,classical,2374,14,77.00,117.24,25.0,41,90+30,artistendo,0,0,4,15,0.0000,0.000000
6,bQZff8t4,2023-04-19,classical,2364,10,43.97,64.34,12.5,32,90+30,artistendo,0,0,2,21,0.0000,0.000000
7,qxwRBbGY,2023-04-19,classical,2364,0,50.33,72.10,9.0,16,90+30,artistendo,0,1,2,29,NaN,NaN
8,h0sbNGGK,2023-04-17,classical,2343,21,154.27,160.15,90.5,30,90+30,artistendo,0,0,2,17,0.0000,0.553973
9,PObmYKgw,2023-04-05,classical,2334,9,26.47,85.99,5.0,62,30+20,artistendo,0,0,1,7,0.1110,0.654189
